In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, split, udf, struct
from pyspark.sql.types import FloatType
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import struct

In [0]:
USERNAME = "big-data"
PASSWORD = "rZ5PCb8ugPiuCm3xCW1oycmH01VNWE"

In [0]:
df = spark.read.parquet("dbfs:/FileStore/Trabalho/transactions.parquet")

In [0]:
def converttoeuro(amount, currency):
    exchangerates = {"USD": 1.05,"GBP": 0.83,"EUR": 1.0,"JPY": 161.36, 'CAD':0.83,'RUB':109.67, 'BRL': 6.34,'AUD': 1.64,'MXN': 21.13,'NGN':1625.22, 'SGD':1.41 }
    return amount * exchangerates.get(currency, 1.0)
converttoeuroudf = udf(converttoeuro, FloatType())
df = df.withColumn("amounteuro", converttoeuroudf(df['amount'], df['currency']))
df = df.withColumn("time", split(col("timestamp"), " ")[1].substr(0, 15))
df = df.withColumn("card_present", col("card_present").cast("int")) \
       .withColumn("is_fraud", col("is_fraud").cast("int")) \
       .withColumn("currency", col("currency").cast("int")) \
       .withColumn("city_size", col("city_size").cast("int")) \
        .withColumn("time", col("time").cast("int")) \
       .withColumn("channel", col("channel").cast("int"))     
df = df.drop('transaction_id','customer_id', 'card_number', 'merchant_category', 
             'merchant_type', 'merchant', 'card_type', 'device', 'device_fingerprint', 
             'high_risk_merchant', 'weekend_transaction','country', "timestamp", 
             "transaction_hour", "city", "ip_address", "velocity_last_hour", "amount")

In [0]:
df

Out[5]: DataFrame[currency: int, city_size: int, card_present: int, channel: int, distance_from_home: bigint, is_fraud: int, amounteuro: float, time: int]

In [0]:
df=df.select(col("currency").cast("int"),col("city_size").cast("int"),col("card_present").cast("string"),col("channel").cast("int"),col("distance_from_home").cast("int"),col("amounteuro").cast("float"), col("time").cast("int"),col("is_fraud").cast("string"))

In [0]:
df_with_parquet = df.withColumn("value", struct([col for col in df.columns]))

In [0]:
df_with_parquet=df_with_parquet.withColumn('value', to_json(struct([col for col in df_with_parquet.columns])))

In [0]:
df_with_parquet = df_with_parquet.withColumn('key', monotonically_increasing_id().cast('string'))

In [0]:
df_with_parquet = df_with_parquet.limit(10)

In [0]:

df_with_parquet.write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "ctvdvd2dt8sjp3bod5rg.any.eu-central-1.mpx.prd.cloud.redpanda.com:9092") \
  .option("kafka.sasl.jaas.config","kafkashaded.org.apache.kafka.common.security.scram.ScramLoginModule required username='{}' password='{}';".format(USERNAME, PASSWORD)) \
  .option("kafka.ssl.endpoint.identification.algmorithm", "https") \
  .option("kafka.security.protocol", "SASL_SSL") \
  .option("kafka.sasl.mechanism", "SCRAM-SHA-512") \
  .option("topic", "project") \
  .save()